In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Objectives

* Apa saja parameter yang mempengaruhi prediksi?

## Metadata Information

* Loan_ID: Nilai unik yang menjelaskan identitas nasabah

* Gender: quite clear --> needs to be encoded

* Married: quite clear --> needs to be encoded

* Dependents: Tanggungan

* Educations: Tingkat pendidikan --> needs to be encoded

* Self_Employed: Wiraswasta atau tidak --> needs to be encoded

* ApplicantIncome: Pendapatan pelamar

* CoapplicantIncome: Penghasilan pelamar bersama

* LoanAmount: Jumlah Pinjaman

* Loan_Amount_Term: Jumlah hari pinjaman

* Credit_History: No information about this feature

* Property_Area: quite clear

* Loan_Status: Target --> needs to be encoded.


# Checking Data

In [ ]:
train_df=pd.read_csv("/kaggle/input/loan-prediction-problem-dataset/train_u6lujuX_CVtuZ9i.csv")

In [ ]:
train_df.head()

In [ ]:
train_df.info()

## Process and Check Null Data

In [ ]:
#null_columns=train_df.columns[train_df.isnull().any()]
#train_df[null_columns].isnull().sum()

train_df.isnull().sum()

In [ ]:
#When inplace = True is used, it performs operation on data and nothing is returned.
#df.some_operation(inplace=True)
#When inplace=False is used, it performs operation on data and returns a new copy of data.
#df = df.an_operation(inplace=False)

# In order to avoid changing datatype, use value = -1 to replace NaN
train_df['Gender'].fillna(value=-1, inplace=True)
train_df['Married'].fillna(value=-1, inplace=True)
train_df['Dependents'].fillna(value=-1, inplace=True)
train_df['Self_Employed'].fillna(value=-1, inplace=True)
train_df['Loan_Amount_Term'].fillna(value=-1, inplace=True)
train_df['Credit_History'].fillna(value=-1, inplace=True)
train_df['LoanAmount'].fillna(value=-1, inplace=True)

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.drop("Loan_ID", axis=1, inplace=True)

In [ ]:
train_df.duplicated().any()

It can be seen that data are not duplicated and all missing values are solved. 

In [ ]:
train_df.describe(include='O')

In [ ]:
train_df.shape

In [ ]:
train_df.head()

In [ ]:
sns.countplot(train_df['Education'])

## Univariate Analysis

In [ ]:
# Split feature as categorical and continuous features

#print(train_df.dtypes)
cat_features=train_df.select_dtypes(exclude=np.number).columns
cont_features=train_df.select_dtypes(include=[np.number,'float64','int64']).columns

features=print(train_df.columns)

#print(cont_features)

In [ ]:
# Exploring categorical data

import matplotlib.pyplot as plt
plt.figure(figsize=(20,8))
for index, col in enumerate(cat_features, start=1):
    plt.subplot(2,4,index)
    plt.title(col)
    plt.pie(train_df[col].value_counts(), labels=train_df[col].value_counts().index)

In [ ]:
# Exploring continuous data

import matplotlib.pyplot as plt
plt.figure(figsize=(20,12))
for index, col in enumerate(cont_features, start=1):
    plt.subplot(2,3,index)
    plt.title(col)
    sns.distplot(train_df[col], kde_kws={'bw':0.5}) # bw adalah bandwith
    #plt.pie(train_df[col].value_counts(), labels=train_df[col].value_counts().index)

In [ ]:
import numpy as np
import seaborn as sns
cont_dummy=cont_features#.drop('store_and_fwd_flag')
plt.figure(figsize=(20,16))
for index, col in enumerate(cont_dummy, start=1):
    plt.subplot(3,5,index)
    plt.title(col)
    ax = sns.boxplot(train_df[col])
    #kde_kws={'bw':0.05})
    #ax.set_yscale('log')

## Bivariate/Multivariate Analysis

In [ ]:
sns.pairplot(train_df, vars=features) #vars = ['mean radius', 'mean texture', 'mean perimeter', 'mean area','mean smoothness'])

In [ ]:
# Exploring categorical data

import matplotlib.pyplot as plt
plt.figure(figsize=(20,12))
for index, col in enumerate(cat_features, start=1):
    plt.subplot(2,4,index)
    plt.title(col)
    #plt.pie(train_df[col].value_counts(), labels=train_df[col].value_counts().index)
    ax=sns.countplot(x=col, hue="Loan_Status", data=train_df )
    # This is annotation
    for p in ax.patches:
        ax.annotate(format(p.get_height(), '.2f'), (p.get_x() + p.get_width() / 2., p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')
#ax = sns.countplot(x="Married", hue="Loan_Status", data=train_df)

# Process Label Encoding

Process label encoding is one of the factors that makes the model of machine learning trained in this program perfoms better results rather than that training model without process label encoding.

In [ ]:
categorical_columns = train_df.select_dtypes(exclude=np.number).columns

train_df = pd.get_dummies(data=train_df, prefix=categorical_columns, drop_first=True)

In [ ]:
train_df.head()

In [ ]:
"""for col in df.columns:
    print(col,':', df[df[col] == '?'][col].count())
    
for cols in df.select_dtypes(exclude=np.number).columns:
    df[cols] = df[cols].str.replace('?', 'Unknown')
    
for cols in df.select_dtypes(exclude=np.number).columns:
    print(cols, ':', df[cols].unique(), end='\n\n')
"""

# Handling Outliers

In [ ]:
"""
from scipy import stats
import numpy as np
z = np.abs(stats.zscore(train_df))
print(z)

threshold = 3
#print(np.where(z > 3))



#print(IQR)
#print(data_df < (Q1 - 1.5 * IQR)) |(data_df > (Q3 + 1.5 * IQR))

train_df2 = train_df[(z < 3).all(axis=1)]

print(train_df2.shape)

train_df2.head()

"""

In [ ]:
"""
#1st way to handling outliers

def get_lower_upper_bound(x):
    q1 = np.percentile(x, 25)
    q3 = np.percentile(x, 75)
    iqr = q3-q1
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)
    return lower_bound, upper_bound

def get_outliers_iqr(x):
    lower_bound, upper_bound = get_lower_upper_bound(x)
    return x[np.where((x > upper_bound) | (x < lower_bound))]

def get_inliers_iqr(x):
    lower_bound, upper_bound = get_lower_upper_bound(x)
    return x[np.where((x < upper_bound) & (x > lower_bound))]


#handling outliers (2nd way)
def ub(s):
    iqr = (np.quantile(s, 0.75))-(np.quantile(s, 0.25))
    upper_bound = np.quantile(s, 0.75)+(1.5*iqr)
    return upper_bound
def lb(s):
    iqr = (np.quantile(s, 0.75))-(np.quantile(s, 0.25))
    lower_bound = np.quantile(s, 0.25)-(1.5*iqr)
    return lower_bound
# function cap
def cap(s):
    s = np.where(s > ub(s), np.median(s), s)
    s = np.where(s < lb(s), np.median(s), s)
    return s
    
#get_outliers_iqr(data_df.values)
"""
#data_df_2 = pd.DataFrame(get_inliers_iqr(data_df.values))     

def cap_outliers(series, zscore_threshold=3, iqr_threshold=1.5, verbose=False, method="IQR"):
    if method=="IQR":
        #https://stackoverflow.com/questions/59489073/how-to-not-remove-but-handle-outliers-by-transforming-using-pandas
        '''Caps outliers to closest existing value within threshold (IQR).'''
        Q1 = series.quantile(0.25)
        Q3 = series.quantile(0.75)
        IQR = Q3 - Q1

        lbound = Q1 - iqr_threshold * IQR
        ubound = Q3 + iqr_threshold * IQR

        outliers = (series < lbound) | (series > ubound)

        series = series.copy()
        series.loc[series < lbound] = series.loc[~outliers].min()
        series.loc[series > ubound] = series.loc[~outliers].max()

        # For comparison purposes.
        if verbose:
                print('\n'.join(
                    ['Capping outliers by the IQR method:',
                     f'   IQR threshold: {iqr_threshold}',
                     f'   Lower bound: {lbound}',
                     f'   Upper bound: {ubound}\n']))
    elif method=="mean":
        '''Caps outliers to closest existing value within threshold (Modified Z-score).'''
        median_val = series.median()
        mad_val = series.mad() # Median absolute deviation

        z_score = (series - median_val) / mad_val
        outliers = abs(z_score) > zscore_threshold

        series = series.copy()
        series.loc[z_score > zscore_threshold] = series.loc[~outliers].max()
        series.loc[z_score < -zscore_threshold] = series.loc[~outliers].min() 

        # For comparison purposes.
        if verbose:
                lbound = median_val - zscore_threshold * mad_val
                ubound = median_val + zscore_threshold * mad_val
                print('\n'.join(
                    ['Capping outliers by the Modified Z-score method:',
                     f'   Z-score threshold: {zscore_threshold}',
                     f'   Lower bound: {lbound}',
                     f'   Upper bound: {ubound}\n']))
    elif method=="zscore":
        '''Caps outliers to closest existing value within threshold (Modified Z-score).'''
        median_val = series.median()
        mad_val = series.mad() # Median absolute deviation

        z_score = (series - median_val) / mad_val
        outliers = abs(z_score) > zscore_threshold

        series = series.copy()
        series.loc[z_score > zscore_threshold] = series.loc[~outliers].mean()
        series.loc[z_score < -zscore_threshold] = series.loc[~outliers].mean() 

        # For comparison purposes.
        if verbose:
                lbound = median_val - zscore_threshold * mad_val
                ubound = median_val + zscore_threshold * mad_val
                print('\n'.join(
                    ['Capping outliers by the Modified Z-score method:',
                     f'   Z-score threshold: {zscore_threshold}',
                     f'   Lower bound: {lbound}',
                     f'   Upper bound: {ubound}\n']))
    elif method=="median":
        '''Caps outliers to closest existing value within threshold (Modified Z-score).'''
        median_val = series.median()
        mad_val = series.mad() # Median absolute deviation

        z_score = (series - median_val) / mad_val
        outliers = abs(z_score) > zscore_threshold

        series = series.copy()
        series.loc[z_score > zscore_threshold] = series.loc[~outliers].max()
        series.loc[z_score < -zscore_threshold] = series.loc[~outliers].min() 

        # For comparison purposes.
        if verbose:
                lbound = median_val - zscore_threshold * mad_val
                ubound = median_val + zscore_threshold * mad_val
                print('\n'.join(
                    ['Capping outliers by the Modified Z-score method:',
                     f'   Z-score threshold: {zscore_threshold}',
                     f'   Lower bound: {lbound}',
                     f'   Upper bound: {ubound}\n']))
    elif method=="zscore":
        '''Caps outliers to closest existing value within threshold (Modified Z-score).'''
        median_val = series.median()
        mad_val = series.mad() # Median absolute deviation

        z_score = (series - median_val) / mad_val
        outliers = abs(z_score) > zscore_threshold

        series = series.copy()
        series.loc[z_score > zscore_threshold] = series.loc[~outliers].median()
        series.loc[z_score < -zscore_threshold] = series.loc[~outliers].median() 

        # For comparison purposes.
        if verbose:
                lbound = median_val - zscore_threshold * mad_val
                ubound = median_val + zscore_threshold * mad_val
                print('\n'.join(
                    ['Capping outliers by the Modified Z-score method:',
                     f'   Z-score threshold: {zscore_threshold}',
                     f'   Lower bound: {lbound}',
                     f'   Upper bound: {ubound}\n']))
    elif method=="log":
        #https://stackoverflow.com/questions/37890849/pandas-series-log-normalize
        series=series.map(lambda x: np.log(x))
        

    return series



features_treating=['ApplicantIncome','CoapplicantIncome','LoanAmount']

for i, col in enumerate(features_treating):
    train_df[col]=cap_outliers(train_df[col],method="IQR", verbose=True)

#train_df['ApplicantIncome']=cap_outliers(train_df["ApplicantIncome"],method="log", verbose=True)
#train_df2 =cap(train_df)


"""# using Z Score
nilai_z = np.abs(scipy.stats.zscore(data_df))
# outliers_df = pd.DataFrame(np.where(outliers>3))
outliers_df = data_df[(nilai_z>2).all(axis=1)]
outliers_df.head()
"""

In [ ]:
train_df.head(100)

In [ ]:
# Exploring continuous data

import matplotlib.pyplot as plt
plt.figure(figsize=(20,12))
for index, col in enumerate(cont_features, start=1):
    plt.subplot(2,3,index)
    plt.title(col)
    sns.distplot(train_df[col], kde_kws={'bw':0.5}) # bw adalah bandwith
    #plt.pie(train_df[col].value_counts(), labels=train_df[col].value_counts().index)

In [ ]:
import numpy as np
import seaborn as sns
cont_dummy=cont_features#.drop('store_and_fwd_flag')
plt.figure(figsize=(20,16))
for index, col in enumerate(cont_dummy, start=1):
    plt.subplot(3,5,index)
    plt.title(col)
    ax = sns.boxplot(train_df[col])
    #kde_kws={'bw':0.05})
    #ax.set_yscale('log')

It seems that the outliers of applicantincome, coapplicantincome and loanamount has been treated successfully.

# Begin training model

In [ ]:
# Encoding categorical features

categorical_columns = train_df.select_dtypes(exclude=np.number).columns
train_df2 = pd.get_dummies(data=train_df, prefix=categorical_columns, drop_first=True)

In [ ]:
train_df2.shape

In [ ]:
pd.set_option('max_columns', 232)
train_df2.head()

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler

X=train_df2.drop('Loan_Status_Y',axis=1) #axis=1 drop the bulk of column
y=train_df2['Loan_Status_Y']

#X=train_df.drop('Loan_Status',axis=1) #axis=1 drop the bulk of column
#y=train_df['Loan_Status']

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.25,random_state=0)

StSc = StandardScaler()
X_train  = StSc.fit_transform(X_train)
X_test  = StSc.fit_transform(X_test)

# Train The Model using Provided Data
## Logistic Regression

In [ ]:
penalty = ['l2', 'elasticnet']
#C = np.logspace(-4,4,20)
C =[0.001,0.01,0.1,1,10,100]
solver=['lbfgs','liblinear']
l1_ratio=[0.001, 0.01, 0.1]

hyperparameters = dict(penalty=penalty, C=C, solver=solver, l1_ratio=l1_ratio)

logreg = LogisticRegression()

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

clf = GridSearchCV(logreg, hyperparameters, cv=cv, verbose=0)



In [ ]:
best_model=clf.fit(X_test,y_test)

In [ ]:
best_model.best_params_

In [ ]:
y_pred=best_model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

print(classification_report(y_test,y_pred))
roc_auc_score(y_test,y_pred)

## Random Forest Classification

In [ ]:
criterion = ['gini','entropy']
max_depth=[5,6,7,8,9]
n_estimator=[50,100,200,300,400,500]

hyperparameters = dict(criterion=criterion, max_depth=max_depth, n_estimators=n_estimator)

randomforest = RandomForestClassifier()

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

clf2 = GridSearchCV(randomforest, param_grid=hyperparameters, cv=cv, verbose=0,n_jobs=-1)

#estimator.get_params().keys()

In [ ]:
best_model2=clf2.fit(X_train, y_train)

In [ ]:
best_model2.best_params_

In [ ]:
y_pred2=best_model2.predict(X_test)

print(classification_report(y_test,y_pred2))
roc_auc_score(y_test,y_pred2)

## XGBoost

In [ ]:
max_depth=[2,3,4,5,6,7,8,9,10]
learning_rate=[0.001,0.01,0.1,0.2,0.3]
min_child_weight=[2,3,4,5,6,7]

hyperparameters = dict(max_depth=max_depth, learning_rate=learning_rate, min_child_weight=min_child_weight)

xgb = XGBClassifier()

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

clf3 = GridSearchCV(xgb, param_grid=hyperparameters, cv=cv, verbose=0,n_jobs=-1)



In [ ]:
#best_model3=clf3.fit(X_train, y_train)
clf3.fit(X_train, y_train)

In [ ]:
import matplotlib.pyplot as plt

print("Feature importance by XGBoost:->\n")
XGBR = XGBClassifier()
XGBR.fit(X,y)
features = XGBR.feature_importances_
Columns = list(X.columns)
for i,j in enumerate(features):
    print(Columns[i],"->",j)
plt.figure(figsize=(16,5))
plt.title(label="XGBC")
plt.bar([x for x in range(len(features))],features)
plt.show()

In [ ]:
y_pred3=clf3.predict(X_test)

print(classification_report(y_test,y_pred3))
roc_auc_score(y_test,y_pred3)

In [ ]:
final_predictions = clf3.predict_proba(X_test)

print(roc_auc_score(y_test, final_predictions[:, 1]))

*Works still on progress*